In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
cols = [ 'expiration_id', 'protocol', 'src_port', 'dst_port',
       'ip_version',   'bidirectional_duration_ms', 'bidirectional_packets',
       'bidirectional_bytes', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'bidirectional_min_ps', 'bidirectional_mean_ps',
       'bidirectional_stddev_ps', 'bidirectional_max_ps',
       'src2dst_min_ps', 'src2dst_mean_ps', 'src2dst_stddev_ps',
       'src2dst_max_ps', 'dst2src_min_ps', 'dst2src_mean_ps',
       'dst2src_stddev_ps', 'dst2src_max_ps', 'bidirectional_min_piat_ms',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'src2dst_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms',
       'src2dst_max_piat_ms', 'dst2src_min_piat_ms',
       'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'bidirectional_syn_packets', 'bidirectional_ack_packets',
       'bidirectional_psh_packets', 'bidirectional_rst_packets',
       'bidirectional_fin_packets', 'src2dst_syn_packets', 'src2dst_ack_packets',
       'src2dst_psh_packets', 'src2dst_rst_packets',
       'src2dst_fin_packets', 'dst2src_syn_packets', 'dst2src_ack_packets',
       'dst2src_psh_packets', 'dst2src_rst_packets',
       'dst2src_fin_packets','application_name',
       'application_category_name', 'application_is_guessed',
       'application_confidence', 'content_type', 'udps.num_pkts_up_to_128_bytes',
       'udps.num_pkts_128_to_256_bytes', 'udps.num_pkts_256_to_512_bytes',
       'udps.num_pkts_512_to_1024_bytes',
       'udps.num_pkts_1024_to_1514_bytes', 'udps.min_ttl', 'udps.max_ttl',
       'udps.min_ip_pkt_len', 'udps.max_ip_pkt_len', 'udps.src2dst_flags',
       'udps.dst2src_flags', 'udps.tcp_flags', 'udps.tcp_win_max_in',
       'udps.tcp_win_max_out', 'udps.icmp_type', 'udps.icmp_v4_type',
       'udps.dns_query_id', 'udps.dns_query_type', 'udps.dns_ttl_answer',
       'udps.ftp_command_ret_code', 'udps.retransmitted_in_packets',
       'udps.retransmitted_out_packets', 'udps.retransmitted_in_bytes',
       'udps.retransmitted_out_bytes', 'udps.src_to_dst_second_bytes',
       'udps.dst_to_src_second_bytes', 'udps.src_to_dst_avg_throughput',
       'udps.dst_to_src_avg_throughput', 'udps.src_to_dst_second_bytes2',
       'udps.dst_to_src_second_bytes2', 'udps.src_to_dst_avg_throughput2',
       'udps.dst_to_src_avg_throughput2', 'udps.tcp_init_ms',
       'udps.tcp_synack_ack_ms', 'udps.tcp_half_closed_time_ms',
       'udps.num_pkts_after_termination',
       'udps.src2dst_first_packet_payload_len',
       'udps.dst2src_first_packet_payload_len',
       'udps.bidirectional_transport_bytes',
       'udps.bidirectional_payload_bytes', 'udps.src2dst_transport_bytes',
       'udps.src2dst_payload_bytes', 'udps.dst2src_transport_bytes',
       'udps.dst2src_payload_bytes',
       'udps.src2dst_most_freq_payload_ratio',
       'udps.src2dst_most_freq_payload_len',
       'udps.dst2src_most_freq_payload_ratio',
       'udps.dst2src_most_freq_payload_len',
       'udps.bidirectional_mean_packet_relative_times',
       'udps.bidirectional_stddev_packet_relative_times',
       'udps.bidirectional_variance_packet_relative_times',
       'udps.bidirectional_coeff_of_var_packet_relative_times',
       'udps.bidirectional_skew_from_median_packet_relative_times',
       'udps.src2dst_mean_packet_relative_times',
       'udps.src2dst_stddev_packet_relative_times',
       'udps.src2dst_variance_packet_relative_times',
       'udps.src2dst_coeff_of_var_packet_relative_times',
       'udps.src2dst_skew_from_median_packet_relative_times',
       'udps.dst2src_mean_packet_relative_times',
       'udps.dst2src_stddev_packet_relative_times',
       'udps.dst2src_variance_packet_relative_times',
       'udps.dst2src_coeff_of_var_packet_relative_times',
       'udps.dst2src_skew_from_median_packet_relative_times',
       'udps.min_req_res_time_diff', 'udps.max_req_res_time_diff',
       'udps.mean_req_res_time_diff', 'udps.stddev_req_res_time_diff',
       'udps.variance_req_res_time_diff',
       'udps.coeff_of_var_req_res_time_diff',
       'udps.skew_from_median_req_res_time_diff',
       'udps.src2dst_small_packet_payload_packets',
       'udps.src2dst_small_packet_payload_ratio',
       'udps.dst2src_small_packet_payload_packets',
       'udps.dst2src_small_packet_payload_ratio',
       'udps.sent_recv_packet_ratio',
       'udps.bidirectional_ps_first_quartile',
       'udps.bidirectional_ps_second_quartile',
       'udps.bidirectional_ps_third_quartile',
       'udps.bidirectional_ps_median_absoulte_deviation',
       'udps.bidirectional_ps_skewness', 'udps.bidirectional_ps_kurtosis',
       'udps.bidirectional_piat_first_quartile',
       'udps.bidirectional_piat_second_quartile',
       'udps.bidirectional_piat_third_quartile',
       'udps.bidirectional_piat_median_absoulte_deviation',
       'udps.bidirectional_piat_skewness',
       'udps.bidirectional_piat_kurtosis',
       'udps.median_req_res_time_diff', 'Attack']

In [2]:
cols0 = [ 'expiration_id', 'protocol',
       'ip_version',   'bidirectional_duration_ms', 'bidirectional_packets',
       'bidirectional_bytes', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'bidirectional_min_ps', 'bidirectional_mean_ps',
       'bidirectional_stddev_ps', 'bidirectional_max_ps',
       'src2dst_min_ps', 'src2dst_mean_ps', 'src2dst_stddev_ps',
       'src2dst_max_ps', 'dst2src_min_ps', 'dst2src_mean_ps',
       'dst2src_stddev_ps', 'dst2src_max_ps', 'bidirectional_min_piat_ms',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'src2dst_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms',
       'src2dst_max_piat_ms', 'dst2src_min_piat_ms',
       'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'bidirectional_syn_packets', 'bidirectional_ack_packets',
       'bidirectional_psh_packets', 'bidirectional_rst_packets',
       'bidirectional_fin_packets', 'src2dst_syn_packets', 'src2dst_ack_packets',
       'src2dst_psh_packets', 'src2dst_rst_packets',
       'src2dst_fin_packets', 'dst2src_syn_packets', 'dst2src_ack_packets',
       'dst2src_psh_packets', 'dst2src_rst_packets',
       'dst2src_fin_packets','application_name',
       'application_category_name', 'application_is_guessed',
       'application_confidence', 'content_type', 'Attack']

In [3]:
def port_feature(port):
    if port < 1024:
        return 1
    elif port < 49152 and port >= 1024:
        return 2
    else:
        return 3        

In [4]:

def encode(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    les = {}
    for each in cols:
        le_col = LabelEncoder()
        df[each] = le_col.fit_transform(df[each])
        les[each] = le_col
       
    return df, les

In [5]:
def normalize_content_type(x):
    if str(x).isspace():
        return "unkown/unkown"
    elif "/" not in str(x):
        return str(x)+"/unkown"
    else:
        return x

In [6]:
def save_predictions(predictions, targets, timeout, save_path= "results_allFeatures/RF"):
    name_file_pred = 'predictions_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"
    name_file_y = 'targets_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"

    pickle.dump(predictions, open(os.path.join(save_path, name_file_pred), 'wb') )
    pickle.dump(targets, open(os.path.join(save_path, name_file_y), 'wb') )
    
def load_predictions(timeout, save_path= "results_allFeatures/RF"):
    name_file_pred = 'predictions_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"
    name_file_y = 'targets_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"
    
    predictions =  pickle.load(open(os.path.join(save_path, name_file_pred), 'rb') )
    targets =  pickle.load(open(os.path.join(save_path, name_file_y), 'rb') )
    return predictions, targets

In [7]:
timeouts = [(0.5,2), (1, 2), (2,2), (0.5,3), (1,3), (2, 3), (3,3), (0.5,4), (1, 4), (2,4), (3,4), (4,4), (0.5,5), (1,5), (2,5), (3,5), (4,5), (5,5), (0.5, 30), (1, 30), (2,30), (3,30), (4,30), (5,30), (10, 30), (0.5, 60), (1, 60), (2,60), (3,60), (4,60), (5,60), (10, 60)]

In [8]:
file = open('../results_allFeatures/RF/cic-ids-2017_results_RF.txt', "w")

In [9]:
def split_data(data):
    train_idx, test_idx = next(StratifiedKFold(n_splits=3).split(data, data['Attack']))
    train, test = data.iloc[train_idx].reset_index(drop=True), data.iloc[test_idx].reset_index(drop=True)
    return train, test

# Training 

In [10]:
best_f1 = 0
best_report = None
best_timeout = None
best_pred = None
best_y = None
for timeout in timeouts:
    print("Processing timeout : ", timeout)
    idle, active = timeout
    out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/project2/extractions/extractions/new_idle_{idle}min_active_{active}min/CIC-IDS-2017'
    df = pd.read_csv(out_dir+"/CIC-IDS-2017.csv")
    file.write("\n==========================================================\n")
    file.write(f" active: {active} min   idle: {idle} min \n")
    file.write(str(df.Attack.value_counts()))
    file.write("\n*******************************************\n")
    file.write(str(df.Attack.value_counts() / df.shape[0] * 100))
    file.write("\n*******************************************\n")
    file.write(str(df['bidirectional_duration_ms'].describe()))
    file.write("\n*******************************************\n")
    df = df[(~df.Attack.str.contains('Attempted')) & (~df.Attack.str.contains('Heartbleed')) ]
    df = df.sort_values(by=['bidirectional_last_seen_ms']).reset_index(drop=True)
    df_new = df[cols]
    df_new['application_name'] = df_new['application_name'].apply(lambda x: x.split(".")[0])
    df_new['content_type'] = df_new['content_type'].fillna("unkown/unkown")
    df_new['content_type'] = df_new['content_type'].apply(lambda x: normalize_content_type(x))
    df_new['content_sub_type'] = df_new['content_type'].apply(lambda x: x.split("/")[1])
    df_new['content_type'] = df_new['content_type'].apply(lambda x: x.split("/")[0])
    df_new['src_port'] = df_new['src_port'].apply(lambda x: port_feature(x))
    df_new['dst_port'] = df_new['dst_port'].apply(lambda x: port_feature(x))
    df_new = df_new.fillna(0)
    categ_cols = ["application_name", "application_category_name", "content_sub_type", "content_type" ]
    df_new, lbl_encoders = encode(df_new,categ_cols)    
    train, test = split_data(df_new)
    y_train = train['Attack']
    X_train = train.drop('Attack', axis=1)
    y_test = test['Attack']
    X_test = test.drop('Attack', axis=1)
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    report = classification_report(y_test, pred,  target_names=le.classes_, digits=4)
    file.write(report)
    save_predictions(y_test, pred, timeout, save_path='results_allFeatures/RF/')
    f1 = f1_score(y_true=y_test, y_pred=pred, average='macro')
    if f1 > best_f1: 
        best_timeout = timeout
        best_f1 = f1
        best_report= report
        best_pred=pred
        best_y=y_test
    file.write("==========================================================\n\n")
    file.flush()
print("--------------- DONE ---------------------")
#save_predictions(best_pred, best_y, save_path='results/RF/')

Processing timeout :  (0.5, 2)
Processing timeout :  (1, 2)
Processing timeout :  (2, 2)
Processing timeout :  (0.5, 3)
Processing timeout :  (1, 3)
Processing timeout :  (2, 3)
Processing timeout :  (3, 3)
Processing timeout :  (0.5, 4)
Processing timeout :  (1, 4)
Processing timeout :  (2, 4)
Processing timeout :  (3, 4)
Processing timeout :  (4, 4)
Processing timeout :  (0.5, 5)
Processing timeout :  (1, 5)
Processing timeout :  (2, 5)
Processing timeout :  (3, 5)
Processing timeout :  (4, 5)
Processing timeout :  (5, 5)
Processing timeout :  (0.5, 30)
Processing timeout :  (1, 30)
Processing timeout :  (2, 30)
Processing timeout :  (3, 30)
Processing timeout :  (4, 30)
Processing timeout :  (5, 30)
Processing timeout :  (10, 30)
Processing timeout :  (0.5, 60)
Processing timeout :  (1, 60)
Processing timeout :  (2, 60)
Processing timeout :  (3, 60)
Processing timeout :  (4, 60)
Processing timeout :  (5, 60)
Processing timeout :  (10, 60)
--------------- DONE ---------------------


In [11]:
file.write("=========================BEST PERFORMANCE===================\n")
file.write(f'best timeouts {best_timeout}\n')
file.write(f'best F1-macro {best_f1}\n')
file.write(best_report)
file.close()

In [12]:
print("best timeouts: ", best_timeout)
print("best F1-macro: ", best_f1)
print("best report: ", best_report)

best timeouts:  (0.5, 60)
best F1-macro:  0.9249687063383619
best report:                              precision    recall  f1-score   support

                    Benign     0.9882    1.0000    0.9941    564579
                    Botnet     1.0000    0.9959    0.9980       245
                      DDoS     1.0000    1.0000    1.0000     30733
             DoS GoldenEye     0.9996    0.9996    0.9996      2633
                  DoS Hulk     1.0000    1.0000    1.0000     52135
          DoS Slowhttptest     1.0000    1.0000    1.0000       961
             DoS Slowloris     0.9989    0.8198    0.9006      2309
               FTP-Patator     0.9962    1.0000    0.9981      1324
              Infiltration     1.0000    0.4286    0.6000        14
   Infiltration - Portscan     0.9999    0.6761    0.8067     20154
                  Portscan     0.9949    1.0000    0.9975     53156
               SSH-Patator     1.0000    0.9554    0.9772       987
  Web Attack - Brute Force     1.0000   

In [13]:
y_pred = best_pred
y_test = best_y

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve


print('\nAccuracy: {:.4f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.4f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.4f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.4f}'.format(f1_score(y_test, y_pred, average='weighted')))

#fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
#print('AUC: {:.4f}'.format(auc(fpr, tpr)))



Accuracy: 0.9904

Micro Precision: 0.9904
Micro Recall: 0.9904
Micro F1-score: 0.9904

Macro Precision: 0.9985
Macro Recall: 0.8837
Macro F1-score: 0.9250

Weighted Precision: 0.9905
Weighted Recall: 0.9904
Weighted F1-score: 0.9895


# Load predictions - Best and Worst case

In [10]:
best_f1 = 0
best_report = None
best_timeout = None
best_pred = None
best_y = None

worst_f1 = 1
worst_report = None
worst_timeout = None
worst_pred = None
worst_y = None


classes = ['Benign', 'Botnet', 'DDoS', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest', 'DoS Slowloris', 'FTP-Patator', 'Infiltration', 'Infiltration - Portscan',
 'Portscan', 'SSH-Patator', 'Web Attack - Brute Force', 'Web Attack - SQL Injection', 'Web Attack - XSS']
for timeout in timeouts:
    print("Loading timeout : ", timeout)
    idle, active = timeout
    pred, y_test = load_predictions(timeout, save_path= "../results_allFeatures/RF")
    report = classification_report(y_test, pred,  target_names=classes, digits=4)
    f1 = f1_score(y_true=y_test, y_pred=pred, average='macro')
    if f1 > best_f1: 
        best_timeout = timeout
        best_f1 = f1
        best_report= report
        best_pred=pred
        best_y=y_test
        
    if f1 <= worst_f1: 
        worst_timeout = timeout
        worst_f1 = f1
        worst_report= report
        worst_pred=pred
        worst_y=y_test
    file.write("==========================================================\n\n")
    file.flush()
print("------------------- DONE -------------------")

Loading timeout :  (0.5, 2)
Loading timeout :  (1, 2)
Loading timeout :  (2, 2)
Loading timeout :  (0.5, 3)
Loading timeout :  (1, 3)
Loading timeout :  (2, 3)
Loading timeout :  (3, 3)
Loading timeout :  (0.5, 4)
Loading timeout :  (1, 4)
Loading timeout :  (2, 4)
Loading timeout :  (3, 4)
Loading timeout :  (4, 4)
Loading timeout :  (0.5, 5)
Loading timeout :  (1, 5)
Loading timeout :  (2, 5)
Loading timeout :  (3, 5)
Loading timeout :  (4, 5)
Loading timeout :  (5, 5)
Loading timeout :  (0.5, 30)
Loading timeout :  (1, 30)
Loading timeout :  (2, 30)
Loading timeout :  (3, 30)
Loading timeout :  (4, 30)
Loading timeout :  (5, 30)
Loading timeout :  (10, 30)
Loading timeout :  (0.5, 60)
Loading timeout :  (1, 60)
Loading timeout :  (2, 60)
Loading timeout :  (3, 60)
Loading timeout :  (4, 60)
Loading timeout :  (5, 60)
Loading timeout :  (10, 60)
------------------- DONE -------------------


In [11]:
file.write("=========================BEST PERFORMANCE===================\n")
file.write(f'best timeouts {best_timeout}\n')
file.write(f'best F1-macro {best_f1}\n')
file.write(f'best resport {best_report}\n')

file.write("=========================WORST PERFORMANCE===================\n")
file.write(f'worst timeouts {worst_timeout}\n')
file.write(f'worst F1-macro {worst_f1}\n')
file.write(f'xorst resport {worst_report}\n')

file.close()

In [12]:
print("best timeouts: ", best_timeout)
print("best F1-macro: ", best_f1)
print("best report: ", best_report)

print("worst timeouts: ", worst_timeout)
print("worst F1-macro: ", worst_f1)
print("worst report: ", worst_report)

best timeouts:  (0.5, 60)
best F1-macro:  0.9249687063383619
best report:                              precision    recall  f1-score   support

                    Benign     1.0000    0.9882    0.9941    571303
                    Botnet     0.9959    1.0000    0.9980       244
                      DDoS     1.0000    1.0000    1.0000     30733
             DoS GoldenEye     0.9996    0.9996    0.9996      2633
                  DoS Hulk     1.0000    1.0000    1.0000     52134
          DoS Slowhttptest     1.0000    1.0000    1.0000       961
             DoS Slowloris     0.8198    0.9989    0.9006      1895
               FTP-Patator     1.0000    0.9962    0.9981      1329
              Infiltration     0.4286    1.0000    0.6000         6
   Infiltration - Portscan     0.6761    0.9999    0.8067     13629
                  Portscan     1.0000    0.9949    0.9975     53425
               SSH-Patator     0.9554    1.0000    0.9772       943
  Web Attack - Brute Force     0.8800   

In [13]:
y_pred =  best_y    # Switched because they were switched when saved
y_test =  best_pred

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve


best_accuracy = accuracy_score(y_test, y_pred)
best_precision = precision_score(y_test, y_pred, average='macro')
best_recall = recall_score(y_test, y_pred, average='macro')

print('\nAccuracy: {:.4f}\n'.format(best_accuracy))

print('Micro Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.4f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.4f}'.format(best_precision))
print('Macro Recall: {:.4f}'.format(best_recall))
print('Macro F1-score: {:.4f}\n'.format(best_f1))

print('Weighted Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.4f}'.format(f1_score(y_test, y_pred, average='weighted')))


Accuracy: 0.9904

Micro Precision: 0.9904
Micro Recall: 0.9904
Micro F1-score: 0.9904

Macro Precision: 0.9985
Macro Recall: 0.8837
Macro F1-score: 0.9250

Weighted Precision: 0.9905
Weighted Recall: 0.9904
Weighted F1-score: 0.9895


In [14]:
y_pred = worst_y
y_test =  worst_pred

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve


worst_accuracy = accuracy_score(y_test, y_pred)
worst_precision = precision_score(y_test, y_pred, average='macro')
worst_recall = recall_score(y_test, y_pred, average='macro')

print('\nAccuracy: {:.4f}\n'.format(worst_accuracy))

print('Micro Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.4f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.4f}'.format(worst_precision))
print('Macro Recall: {:.4f}'.format(worst_recall))
print('Macro F1-score: {:.4f}\n'.format(worst_f1))

print('Weighted Precision: {:.4f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.4f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.4f}'.format(f1_score(y_test, y_pred, average='weighted')))


Accuracy: 0.9873

Micro Precision: 0.9873
Micro Recall: 0.9873
Micro F1-score: 0.9873

Macro Precision: 0.9314
Macro Recall: 0.8164
Macro F1-score: 0.8480

Weighted Precision: 0.9875
Weighted Recall: 0.9873
Weighted F1-score: 0.9859


In [15]:
file_best_worst = open('../results_allFeatures/RF/cic-ids-2017_best&worst.txt', "w")

file_best_worst.write(f'best timeouts:  {best_timeout}\n')
file_best_worst.write(f'best F1-macro:  {best_f1}\n')
file_best_worst.write(f'best precision:  {best_precision}\n')
file_best_worst.write(f'best recall:  {best_recall}\n')
file_best_worst.write(f'best accuracy:  {best_accuracy}\n')
file_best_worst.write(f'best report:  {best_report}\n')

file_best_worst.write(f'worst timeouts:  {worst_timeout}\n')
file_best_worst.write(f'worst F1-macro:  {worst_f1}\n')
file_best_worst.write(f'worstt precision:  {worst_precision}\n')
file_best_worst.write(f'worst recall:  {worst_recall}\n')
file_best_worst.write(f'worst accuracy:  {worst_accuracy}\n')
file_best_worst.write(f'worst report:  {worst_report}\n')
file_best_worst.flush()
file_best_worst.close

<function TextIOWrapper.close()>